In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from Lenet import LeNet5
from torchvision import transforms, datasets

from torchmetrics import Accuracy
from torchinfo import summary



In [2]:
# Normalization transform
transform_normalize = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform_normalize)

In [3]:
indices_of_zeros = [i for i, label in enumerate(mnist_dataset.targets) if label == 0]

# Randomly select 100-200 indices of '0' digits
# You can adjust the number by changing the value of num_samples
num_samples = 100  # or 200, depending on your requirement
selected_indices = np.random.choice(indices_of_zeros, num_samples, replace=False)

# Create a subset from the MNIST dataset using the selected indices
subset_of_zeros = torch.utils.data.Subset(mnist_dataset, selected_indices)

# Verify the dataset
print(f"Number of images in the subset: {len(subset_of_zeros)}")


Number of images in the subset: 100


In [15]:
zero_loader = torch.utils.data.DataLoader(subset_of_zeros, batch_size=10, shuffle=True, num_workers=0)


In [11]:
pre_trained_model = LeNet5(num_classes=9)

pre_trained_model.load_state_dict(torch.load('lenet_1_to_9_v2.pth'))

<All keys matched successfully>

In [13]:
# Modify the last layer of the classifier to output 10 classes
pre_trained_model.classifier[5] = nn.Linear(in_features=84, out_features=10, bias=True)


In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pre_trained_model.classifier[5].parameters(), lr=1e-4)

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

accuracy = Accuracy(task='multiclass', num_classes=10)
accuracy = accuracy.to(device)

pre_trained_model = pre_trained_model.to(device)

In [24]:
num_epochs = 25 

for epoch in range(num_epochs):
    pre_trained_model.train()
    train_loss, train_acc = 0, 0

    for images, labels in zero_loader:

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad() # Zero the gradients

        outputs = pre_trained_model(images) # Forward pass

        loss = criterion(outputs, labels) # Calculate the loss
        acc = accuracy(outputs, labels)

        loss.backward() # Backward pass

        optimizer.step() # Update the weights

        train_loss += loss.item()
        train_acc += acc

    train_loss /= len(zero_loader)
    train_acc /= len(zero_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

Epoch 1/25, Loss: 1.2428, Accuracy: 0.9500
Epoch 2/25, Loss: 1.1982, Accuracy: 0.9800
Epoch 3/25, Loss: 1.1553, Accuracy: 0.9900
Epoch 4/25, Loss: 1.1133, Accuracy: 0.9900
Epoch 5/25, Loss: 1.0729, Accuracy: 0.9900
Epoch 6/25, Loss: 1.0342, Accuracy: 0.9900
Epoch 7/25, Loss: 0.9964, Accuracy: 0.9900
Epoch 8/25, Loss: 0.9602, Accuracy: 0.9900
Epoch 9/25, Loss: 0.9255, Accuracy: 0.9900
Epoch 10/25, Loss: 0.8921, Accuracy: 0.9900
Epoch 11/25, Loss: 0.8597, Accuracy: 0.9900
Epoch 12/25, Loss: 0.8289, Accuracy: 0.9900
Epoch 13/25, Loss: 0.7992, Accuracy: 0.9900
Epoch 14/25, Loss: 0.7708, Accuracy: 0.9900
Epoch 15/25, Loss: 0.7435, Accuracy: 0.9900
Epoch 16/25, Loss: 0.7173, Accuracy: 0.9900
Epoch 17/25, Loss: 0.6924, Accuracy: 1.0000
Epoch 18/25, Loss: 0.6683, Accuracy: 1.0000
Epoch 19/25, Loss: 0.6454, Accuracy: 1.0000
Epoch 20/25, Loss: 0.6232, Accuracy: 1.0000
Epoch 21/25, Loss: 0.6023, Accuracy: 1.0000
Epoch 22/25, Loss: 0.5824, Accuracy: 1.0000
Epoch 23/25, Loss: 0.5630, Accuracy: 1.00